# Requirement 5

In [56]:
import numpy as np

np.set_printoptions(precision=3)

import numpy as np

class PricingEnvConversion:
    def __init__(self, mu, T, conversion_fn, n_customers=100):
        """
        mu: N x P matrix of per-unit profit (price - cost)
        I: total cumulative inventory across all rounds
        T: total number of rounds
        conversion_fn: function which for a given price index gives buying probability
        n_customers: number of potential buyers per round
        """
        self.mu = np.array(mu)
        self.T = T
        self.conversion_fn = conversion_fn
        self.n_customers = n_customers
        self.t = 0
        self.N, self.P = self.mu.shape

    def round(self, price_vector):
        if self.t >= self.T:
            raise Exception("Environment finished all rounds.")

        rewards_per_unit = np.zeros(self.N)
        demand = np.zeros(self.N, dtype=int)
        profit = np.zeros(self.N)

        for i in range(self.N):
            price_idx = price_vector[i]
            if price_idx == -1:
                # skip this product
                continue
            # compute reward and demand
            rewards_per_unit[i] = self.mu[i, price_idx]
            prob = self.conversion_fn(price_idx)
            demand[i] = np.random.binomial(self.n_customers, prob)
            profit[i] = rewards_per_unit[i] * demand[i]

        self.t += 1
        return demand, profit


In [57]:
from scipy import optimize


class CombinatorialUCB:
    def __init__(self, N, P, B, T):
        """
        N: number of products
        P: number of prices
        B: shared capacity (not used right now)
        """
        self.N = N
        self.P = P
        self.B = B
        self.remaining_inventory = B
        self.T = T
        self.counts = np.zeros((N, P))  # times each arm is chosen
        self.means = np.zeros((N, P))   # estimated mean rewards
        self.avg_c = np.zeros((N, P))
        self.t = 0

    def pull_arm(self):
        prices = []

        # If inventory exhausted, sell nothing
        if self.remaining_inventory < 1:
            prices = [-1] * self.N
            return prices

        if self.t < self.P:
            prices = [self.t % self.P] * self.N
            return prices

        f_ucbs = self.means + np.sqrt(2*np.log(self.t + 1)/(self.counts + 1e-6))
        c_lcbs = self.avg_c - np.sqrt(2*np.log(self.t + 1)/(self.counts + 1e-6))

        rho = self.remaining_inventory / max(1, self.T - self.t)

        # compute LP to get probabilities over product/price pairs
        gamma = self.compute_opt(f_ucbs, c_lcbs, rho)

        for i in range(self.N):
            probs = np.clip(gamma[i], 0, None)
            probs /= probs.sum()

            price_idx = np.random.choice(len(probs), p=probs)
            
            # If the sampled index is 0, treat it as "do not sell"
            if price_idx == 0:
                prices.append(-1)
            else:
                prices.append(price_idx - 1)  # shift back to match original price indices

        return prices

    
    def compute_opt(self, f_ucbs, c_lcbs, rho):
        """
        Args:
        f_ucbs: (N x P) upper confidence bounds on expected rewards
        c_lcbs: (N x P) lower confidence bounds on expected consumptions
        rho: per-round budget (remaining_inventory / remaining_time)

        Returns:
            gamma: (N x (P+1)) array of probabilities over price choices
        """
        N, P = self.N, self.P

        # Add dummy price (0 reward, 0 cost) as first column (it's equivalent to "do not sell")
        f_ucbs = np.hstack([np.zeros((N, 1)), f_ucbs])
        c_lcbs = np.hstack([np.zeros((N, 1)), c_lcbs])

        # Objective: maximize expected reward
        c_obj = -f_ucbs.flatten()

        # Bounds: gamma_ij >= 0
        bounds = (0, 1)

        # Equality constraints: sum of each row = 1
        A_eq = np.zeros((N, N * (P + 1)))
        b_eq = np.ones(N)
        for i in range(N):
            for j in range(P + 1):
                A_eq[i, i * (P + 1) + j] = 1

        # Inequality constraint: expected cost <= rho
        A_ub = [c_lcbs.flatten()]
        b_ub = [rho]

        res = optimize.linprog(c=c_obj, A_ub=A_ub, b_ub=b_ub, A_eq=A_eq, b_eq=b_eq, bounds=bounds)

        if res.success:
            gamma = res.x.reshape(N, P + 1)
        else:
            gamma = np.zeros((N, P + 1))

        return gamma



    def update(self, prices, demand, rewards):
        """
        prices: list of chosen price indices per product (-1 if product not sold)
        demand: list of units sold per product
        rewards: observed per-product reward
        """
        self.t += 1

        for i in range(self.N):
            p = prices[i]
            if p == -1:
                continue  # product not sold, skip

            units_sold = demand[i]

            # Update inventory
            self.remaining_inventory -= units_sold
            if self.remaining_inventory < 0:
                self.remaining_inventory = 0  # inventory can't go negative

            # Update estimated mean reward for this product/price pair
            self.counts[i, p] += 1
            n = self.counts[i, p]
            self.means[i, p] += (rewards[i] - self.means[i, p]) / n

            self.avg_c[i, p] += (units_sold - self.avg_c[i, p]) / n



In [58]:
def clairvoyant_sample_action(gamma, f):
    """
    Args:
        gamma: (N x (P+1)) array of action probabilities from the LP,
               where the first column is the "do not sell" option.
        f: (N x P) array of true expected rewards for each product/price pair.

    Returns:
        sampled_action: list of chosen price indices per product
                        (-1 indicates "do not sell").
        expected_reward_per_round: float, sum of expected rewards across products
                                   for the sampled action (already accounts for n_customers)
    """

    N, _ = gamma.shape

    sampled_action = []
    expected_reward_per_round = 0.0
    for i in range(N):
        probs = np.clip(gamma[i], 0, None)
        if probs.sum() > 0:
            probs /= probs.sum()
        else:
            probs = np.zeros_like(probs)
            probs[0] = 1  # fallback: "do not sell"
        idx = np.random.choice(len(probs), p=probs)
        if idx == 0:
            sampled_action.append(-1)
        else:
            sampled_action.append(idx-1)
            expected_reward_per_round += f[i, idx-1]

    return sampled_action, expected_reward_per_round

In [59]:
from collections import deque


class DualAgent:
    def __init__(self, N, P, B, T, window_size=100):
        """
        N: number of products
        P: number of prices
        B: shared capacity (total inventory)
        T: total number of rounds
        window_size: number of past rounds to keep in the sliding window
        """
        self.N = N
        self.P = P
        self.B = B
        self.T = T
        self.window_size = window_size

        self.lambda_ = 0

        self.reset()

    def reset(self):
        """Reset agent state for a new simulation"""
        self.remaining_inventory = self.B
        self.t = 0
        # Sliding window: store last window_size rounds
        self.rewards_window = deque(maxlen=self.window_size)       
        self.consumptions_window = deque(maxlen=self.window_size)  

        self.lambda_ = 0

    def pull_arm(self):
        """
        Pull arm using primal-dual approach with sliding window statistics.
        
        Parameters
        ----------
        lam : float
            Current dual variable λ (shadow price of capacity)
        
        Returns
        -------
        prices : list of int
            Selected price index for each product (-1 = not offered)
        """
        if self.remaining_inventory < 1:
            return [-1] * self.N

        # --- Compute empirical means and average consumptions from sliding window ---
        if len(self.rewards_window) == 0:
            means = np.zeros((self.N, self.P))
            avg_c = np.zeros((self.N, self.P))
            counts = np.zeros((self.N, self.P))
        else:
            rewards_stack = np.stack(self.rewards_window, axis=0)       # shape: window_size x N x P
            consumptions_stack = np.stack(self.consumptions_window, axis=0)
            counts = np.sum(rewards_stack != 0, axis=0)
            means = np.sum(rewards_stack, axis=0) / np.maximum(1, counts)
            avg_c = np.sum(consumptions_stack, axis=0) / np.maximum(1, counts)

        # --- Initial exploration: pick different price for each product ---
        if self.t < self.P:
            return [self.t % self.P for _ in range(self.N)]

        # --- Primal-dual step: compute optimal probability distribution ---
        # Use empirical means for reward and consumption, and current lambda
        if(self.t % 10 == 0):
            print("My current means is")
            print(means)
            print("My current avg_c is")
            print(avg_c)
            print("My current lambda is")
            print(self.lambda_)

        gamma = self.compute_opt(means, avg_c, self.lambda_)

        # --- Sample prices for each product from computed distribution ---
        prices = []
        for i in range(self.N):
            probs = np.clip(gamma[i], 0, None)
            probs /= probs.sum() + 1e-8  # normalize to make it a valid distribution
            choice = np.random.choice(len(probs), p=probs)
            prices.append(-1 if choice == 0 else choice - 1)  # 0 = not offered

        return prices
        
    def compute_opt(self, f_means, c_means, lambda_):
        """
        Compute optimal product/price distribution under primal-dual Lagrangian.

        Parameters
        ----------
        f_means : np.ndarray, shape (N, P)
            Empirical mean rewards for each product i and price option j
        c_means : np.ndarray, shape (N, P)
            Empirical mean consumptions (units) for each product i and price option j
        lam : float
            Current dual variable λ (shadow price of capacity)

        Returns
        -------
        policy : np.ndarray, shape (N, P+1)
            Probability distribution over actions for each product.
            Column 0 corresponds to "not offering the product".
        """
        N, P = self.N, self.P

        # Add dummy "not offering" option (zero reward, zero cost)
        f_means = np.hstack([np.zeros((N, 1)), f_means])
        c_means = np.hstack([np.zeros((N, 1)), c_means])

        # Build objective coefficients: reward minus λ * cost
        obj = (f_means - lambda_ * c_means).flatten()

        # Constraints: each product must choose exactly one action
        A_eq = np.zeros((N, N * (P + 1)))
        b_eq = np.ones(N)
        for i in range(N):
            for j in range(P + 1):
                A_eq[i, i * (P + 1) + j] = 1

        # Bounds: each variable is in [0,1]
        bounds = [(0, 1) for _ in range(N * (P + 1))]

        # Solve LP (maximize linear function → minimize negative)
        res = optimize.linprog(
            c=-obj,
            A_eq=A_eq, b_eq=b_eq,
            bounds=bounds,
            method="highs"
        )

        if res.success:
            return res.x.reshape(N, P + 1)
        else:
            # fallback: uniform distribution over all options (including "not offer")
            return np.ones((N, P + 1)) / (P + 1)
        

    def update(self, prices, demand, rewards):
        """
        Update sliding-window estimates and dual variable with observed data.

        Arguments:
            prices: list of chosen price indices per product (-1 if product not offered)
            demand: observed demand per product
            rewards: observed per-product reward
        """
        self.t += 1

        # --- Update remaining inventory ---
        total_consumed = sum(demand)
        self.remaining_inventory = max(0, self.remaining_inventory - total_consumed)

        # --- Construct this round's matrices for sliding window ---
        rewards_mat = np.zeros((self.N, self.P))
        consumption_mat = np.zeros((self.N, self.P))
        for i in range(self.N):
            p = prices[i]
            if p != -1:
                rewards_mat[i, p] = rewards[i]
                consumption_mat[i, p] = demand[i]

        # --- Append to sliding window ---
        self.rewards_window.append(rewards_mat)
        self.consumptions_window.append(consumption_mat)

        # --- Update dual variable lambda (primal-dual) ---
        # rho = per-round budget allowance
        rho = self.remaining_inventory / max(1, self.T - self.t)
        eta = 0.1 / self.N
        self.lambda_ = max(0, self.lambda_ + eta * (total_consumed - rho))

In [60]:
import numpy as np

# environment setup
N, P = 3, 5
B, T = 250_000, 10_000
n_customers = 20
#n_trials = 10
n_trials = 1

mu = np.array([[1, 2, 3, 4, 5],
               [2, 3, 4, 5, 6],
               [1, 3, 5, 7, 9]])

conversion_fn = lambda price_idx: max(0, 1 - 0.2*price_idx)

# Cumulative regret
all_cum_regret_ratios = np.zeros((n_trials, T))

# Expected reward per product/price per customer
f = mu * np.array([conversion_fn(p) for p in range(P)]) * n_customers

# Cost per product/price (for all customers)
c = np.array([[conversion_fn(p) for p in range(P)] for _ in range(N)]) * n_customers

agent_clar = CombinatorialUCB(N, P, B, T)
rho_clar = B / T

# LP solution: probability distribution over prices
gamma_clar = agent_clar.compute_opt(f, c, rho_clar)

for trial in range(n_trials):
    print(f"\n=== Starting Trial {trial+1} ===")

    #agent = CombinatorialUCB(N, P, B, T)
    agent = DualAgent(N=N, P=P, B=B, T=T, window_size=99999)
    env = PricingEnvConversion(mu, T, conversion_fn, n_customers=n_customers)

    total_agent_reward = 0
    total_clair_reward = 0

    for t in range(T):
        # Agent step
        agent_prices = agent.pull_arm()
        demand, rewards = env.round(agent_prices)
        agent.update(agent_prices, demand, rewards)
        total_agent_reward += rewards.sum()

        # Clairvoyant step
        clair_action, clair_reward_per_round = clairvoyant_sample_action(gamma_clar, f)
        total_clair_reward += clair_reward_per_round

        # Compute cumulative regret ratio
        cumulative_regret = total_clair_reward - total_agent_reward
        all_cum_regret_ratios[trial, t] = cumulative_regret / (t + 1)

        # Print checkpoints every 5 rounds
        if (t + 1) % 5 == 0:
            print(f"Trial {trial+1}, Round {t+1}:")
            print(f"  Clairvoyant action: {clair_action}, reward per round: {clair_reward_per_round:.2f}")
            print(f"  Agent action:       {agent_prices}, reward this round: {rewards.sum():.2f}")
            print(f"  Cumulative clairvoyant reward: {total_clair_reward:.2f}")
            print(f"  Cumulative agent reward:       {total_agent_reward:.2f}")
            print(f"  Cumulative regret: {cumulative_regret:.2f}")
            print(f"  Regret / T ratio:  {cumulative_regret / (t+1):.2f}")
            print("--------------------------------------------------")

# Final cumulative regret ratio per trial
final_ratios = all_cum_regret_ratios[:, -1]
print("\n--- Final regret/T ratio per trial ---")
for trial in range(n_trials):
    print(f"Trial {trial+1}: regret/T = {final_ratios[trial]:.2f}")
print(f"Average final regret/T over {n_trials} trials: {final_ratios.mean():.2f}")


=== Starting Trial 1 ===
Trial 1, Round 5:
  Clairvoyant action: [3, 3, 3], reward per round: 128.00
  Agent action:       [4, 4, 4], reward this round: 84.00
  Cumulative clairvoyant reward: 656.00
  Cumulative agent reward:       594.00
  Cumulative regret: 62.00
  Regret / T ratio:  12.40
--------------------------------------------------
Trial 1, Round 10:
  Clairvoyant action: [3, 3, 3], reward per round: 128.00
  Agent action:       [3, 3, 3], reward this round: 127.00
  Cumulative clairvoyant reward: 1312.00
  Cumulative agent reward:       1176.00
  Cumulative regret: 136.00
  Regret / T ratio:  13.60
--------------------------------------------------
My current means is
[[20.    34.    33.    35.333 15.   ]
 [40.    42.    46.667 46.25   6.   ]
 [20.    48.    60.    57.75  45.   ]]
My current avg_c is
[[20.    17.    11.     8.833  3.   ]
 [20.    14.    11.667  9.25   1.   ]
 [20.    16.    12.     8.25   5.   ]]
My current lambda is
1.7351576610134931
Trial 1, Round 15:
  